In [1]:
import pandas as pd
import numpy as np
import requests
import tweepy
import json
import os
import re
from IPython.display import clear_output

------
<a id="indice"></a>

# Índice

1. **[Coleta](#coleta)**
    * [Twitter Archive](#coleta:twitter-archive)
    * [Image Prediction](#coleta:image-prediction)
    * [Twitter API](#coleta:twitter-api)
    
    
2. **[Avaliação](#avaliacao)**
    * [Twitter Archive](#avaliacao:twitter-archive)
    * [Image Prediction](#avaliacao:image-prediction)
    * [Twitter API](#avaliacao:twitter-api)
    * [Notas da avaliação](#avaliacao:anotacoes)
    
    
3. **[Limpeza](#limpeza)**
    * [Definição](#limpeza:definicao)
    * [Twitter Archive](#limpeza:twitter-archive)
    * [Image Prediction](#limpeza:image-prediction)
    * [Twitter API](#limpeza:twitter-api)


4. **[Armazenamento](#armazenamento)**

------
<a id="coleta"></a>

# Coleta

<a id="coleta:twitter-archive"></a>

## Coleta: Twitter archive

In [2]:
df_twitter_arc = pd.read_csv('data/twitter-archive-enhanced.csv')
df_twitter_arc.sample(2)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
1410,699036661657767936,NaN,NaN,2016-02-15 01:05:02 +0000,"<a href=""http://twitter.com/download/iphone"" r...",HAPPY V-DAY FROM YOUR FAV PUPPER SQUAD 13/10 f...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/699036661...,13,10,None,None,None,pupper,None
712,784431430411685888,NaN,NaN,2016-10-07 16:33:21 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Stormy. He's curly af. Already pupared...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/784431430...,12,10,Stormy,None,None,None,None


<a id="coleta:image-prediction"></a>

## Coleta: Image prediction

In [3]:
df_prediction = None

r = requests.get('https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv')

if r.status_code is 200:
    df_prediction = pd.read_csv(pd.compat.StringIO(r.text), sep='\t')    
else:    
    print('ERROR: Image prediction request returned {status_code} status code.'.format(status_code = r.status_code))

In [4]:
df_prediction.sample(2)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
317,671744970634719232,https://pbs.twimg.com/media/CVKEfMKWoAAR-Ud.jpg,1,ice_bear,0.251193,False,ram,0.213839,False,Arctic_fox,0.081551,False
1924,857263160327368704,https://pbs.twimg.com/media/C-WcS4MXoAADrBU.jpg,1,Samoyed,0.998021,True,Pomeranian,0.000922,True,keeshond,0.000311,True


<a id="coleta:twitter-api"></a>

## Coleta: Twitter API

In [5]:
# Carregar configuração do Twitter App
with open('twitter_config.json', 'r', encoding='utf-8') as file:
    app_config = json.load(file)

In [6]:
# Atribuição da configurações a variaveis locais
api_key = app_config['api_key']
api_secret = app_config['api_secret']
access_token = app_config['access_token']
access_secret = app_config['access_secret']

In [7]:
# Autentificação ao Twitter App
auth = tweepy.OAuthHandler(api_key, api_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

Buscar dados do histórico de Tweets na API

In [8]:
json_path = 'data/tweet_json.txt'
error_log = 'data/tweet_error.log'

# Salvar Tweets localmente
if(not os.path.isfile(json_path)):
    print('Aguarde. Dado o volume de requisições, a coleta de tweets pode demorar alguns minutos.')
    tweet_count = 0
    for tweet_id in df_twitter_arc.tweet_id:
        print('{percent}%'.format(percent=int((++tweet_count/df_twitter_arc.tweet_id.size)*100)))
        try:
            status = api.get_status(tweet_id)
            with open(json_path, 'a', newline='\n') as file:
                file.write(f'{json.dumps(status._json)}\n')

        except Exception as err:
            with open(error_log, 'a', newline='\n') as log:
                log.write(f'{str(tweet_id)}: {str(err.args[0])}\n')
            print(f'{str(tweet_id)}: {str(err.args[0])}')
        
        clear_output(wait=True)
else:
    print('Dados já salvos em disco, não serão executadas novas requisições à API do Twitter.')
        

Dados já salvos em disco, não serão executadas novas requisições à API do Twitter.


In [9]:
# Criar uma lista de Dictionaries com os Tweets carregados
tweets = []

with open(json_path, 'r') as file:
    for line in file:
        try:
            tweet = json.loads(line)
            
            if(tweet.get('entities', False)):
                if(tweet['entities'].get('media', False) and tweet['entities']['media'][0].get('media_url', False)):
                    tweets.append({
                        'id': int(tweet['id']),
                        'created_at': tweet['created_at'],
                        'in_reply_to_status_id': str(tweet['in_reply_to_status_id']),
                        'in_reply_to_user_id': str(tweet['in_reply_to_user_id']),
                        'is_quote_status': tweet['is_quote_status'],
                        'retweet_count': int(tweet['retweet_count']),
                        'favorite_count': int(tweet['favorite_count']),
                        'media_url': tweet['entities']['media'][0]['media_url'],
                        'retweeted': tweet['retweeted'],
                        'favorited': tweet['favorited']
                    })
            
            
        except Exception as e:
            print(e) 
            
        

In [10]:
# Criar dataframe de Tweets consultados na API
columns = tweets[0].keys()
df_tweets_api = pd.DataFrame(tweets, columns = columns)

------
<a id="avaliacao"></a>

# Avaliação

<a id="avaliacao:twitter-archive"></a>

## Avaliação: Twitter data archive

In [11]:
 df_twitter_arc.sample(5)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
661,791026214425268224,NaN,NaN,2016-10-25 21:18:40 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: This is Alfie. He's touching a ...,7.638376e+17,4.196984e+09,2016-08-11 20:40:41 +0000,https://twitter.com/dog_rates/status/763837565...,11,10,Alfie,None,None,None,None
27,886736880519319552,NaN,NaN,2017-07-16 23:58:41 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Mingus. He's a wonderful father to his...,NaN,NaN,NaN,"https://www.gofundme.com/mingusneedsus,https:/...",13,10,Mingus,None,None,None,None
576,800859414831898624,8.008580e+17,2.918590e+08,2016-11-22 00:32:18 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@SkyWilliams doggo simply protecting you from ...,NaN,NaN,NaN,NaN,11,10,None,doggo,None,None,None
1390,700151421916807169,NaN,NaN,2016-02-18 02:54:41 +0000,"<a href=""http://twitter.com/download/iphone"" r...",If a pupper gave that to me I'd probably start...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/700151421...,11,10,None,None,None,pupper,None
1689,681340665377193984,6.813394e+17,4.196984e+09,2015-12-28 05:07:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",I've been told there's a slight possibility he...,NaN,NaN,NaN,NaN,5,10,None,None,None,None,None


In [12]:
df_twitter_arc.describe()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,retweeted_status_id,retweeted_status_user_id,rating_numerator,rating_denominator
count,2.356000e+03,7.800000e+01,7.800000e+01,1.810000e+02,1.810000e+02,2356.000000,2356.000000
mean,7.427716e+17,7.455079e+17,2.014171e+16,7.720400e+17,1.241698e+16,13.126486,10.455433
std,6.856705e+16,7.582492e+16,1.252797e+17,6.236928e+16,9.599254e+16,45.876648,6.745237
min,6.660209e+17,6.658147e+17,1.185634e+07,6.661041e+17,7.832140e+05,0.000000,0.000000
25%,6.783989e+17,6.757419e+17,3.086374e+08,7.186315e+17,4.196984e+09,10.000000,10.000000
50%,7.196279e+17,7.038708e+17,4.196984e+09,7.804657e+17,4.196984e+09,11.000000,10.000000
75%,7.993373e+17,8.257804e+17,4.196984e+09,8.203146e+17,4.196984e+09,12.000000,10.000000
max,8.924206e+17,8.862664e+17,8.405479e+17,8.874740e+17,7.874618e+17,1776.000000,170.000000


In [13]:
df_twitter_arc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

<a id="twitter-arc:info"></a>
* Dados incompletos, contém apenas 2356 registros dos 5000 anunciados
* `in_reply_to_status_id`, `in_reply_to_user_id`, `retweeted_status_id` e `retweeted_status_user_id` em `float`
* `timestamp` e `retweeted_status_timestamp` em `string`

In [14]:
# Verificar se há IDs duplicados
df_twitter_arc[df_twitter_arc.tweet_id.duplicated()]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo


<a id="twitter-arc:source"></a>

In [15]:
# Visualizar exemplos de dados da coluna `source`
df_twitter_arc.loc[:, 'source'].sample(10)

1726    <a href="https://about.twitter.com/products/tw...
1242    <a href="http://twitter.com/download/iphone" r...
2164    <a href="http://twitter.com/download/iphone" r...
1439    <a href="http://twitter.com/download/iphone" r...
1336    <a href="http://twitter.com/download/iphone" r...
1914    <a href="http://twitter.com/download/iphone" r...
2091    <a href="http://twitter.com/download/iphone" r...
1867    <a href="http://twitter.com/download/iphone" r...
1498    <a href="http://twitter.com/download/iphone" r...
2341    <a href="http://twitter.com/download/iphone" r...
Name: source, dtype: object

Variável `source` é uma URL para a Aplicação utilizada no tweet, sem valor para a unidade de observação

<a id="twitter-arc:name"></a>

In [16]:
# Visualizar exemplos da coluna `name`
df_twitter_arc.name.sample(10)

1205    Bubbles
682         mad
1676       None
635       Dobby
1177      Clyde
1241    Chester
1868     Arnold
571     Wallace
336       Orion
1025         an
Name: name, dtype: object

In [17]:
df_twitter_arc[df_twitter_arc.name == 'None'].name.count()

745

* Nomes (`name`) preenchidos com valor literal igual a `'None'`, supostamente um valor não preenchido, vazio

<a id="twitter-arc:category"></id>

In [18]:
# Visualizar exemplo de dados das colunas 'doggo' a 'puppo'
df_twitter_arc.loc[:,'doggo':].sample(10)

,doggo,floofer,pupper,puppo
1931,None,None,None,None
1203,None,None,None,None
1127,None,None,None,None
2304,None,None,None,None
297,None,None,pupper,None
1808,None,None,None,None
2269,None,None,None,None
1044,None,None,None,None
263,None,None,None,None
467,None,None,None,None


* Valores não preenchidos como 'None' literal

Uma variável em várias colunas

In [19]:
# Verificar padrão no preenchimento das classificações
df_twitter_arc.loc[:,'doggo':].nunique()

doggo      2
floofer    2
pupper     2
puppo      2
dtype: int64

In [20]:
df_twitter_arc.loc[:, 'doggo':'puppo'].isna().sum()

doggo      0
floofer    0
pupper     0
puppo      0
dtype: int64

<a id="twitter-arc:expanded_urls"></a>

Exemplos da coluna `expanded_urls`

In [21]:
df_twitter_arc.loc[:, 'expanded_urls'].sample(20)

1515                        https://vine.co/v/iOZKZEU2nHq
1628                        https://vine.co/v/ihWIxntjtO7
1080                                                  NaN
1411    https://twitter.com/dog_rates/status/698989035...
273     https://twitter.com/dog_rates/status/667152164...
1200    https://twitter.com/chpsanfrancisco/status/716...
150     https://www.gofundme.com/helpquinny,https://tw...
2023    https://twitter.com/dog_rates/status/671896809...
589     https://twitter.com/dog_rates/status/774314403...
2238    https://twitter.com/dog_rates/status/668113020...
301     https://twitter.com/dog_rates/status/836677758...
224     https://twitter.com/dog_rates/status/849336543...
806     https://twitter.com/dog_rates/status/772102971...
1541    https://twitter.com/dog_rates/status/689623661...
1229    https://twitter.com/dog_rates/status/713761197...
1191    https://twitter.com/dog_rates/status/717841801...
750     https://twitter.com/dog_rates/status/779123168...
2319    https:

In [22]:
df_twitter_arc.loc[:, 'expanded_urls'].isna().value_counts()

False    2297
True       59
Name: expanded_urls, dtype: int64

* Registros de `expanded_urls` com URLs para os twittes originais. Não apresenta valor a unidade em observacão

------
<a id="avaliacao:image-prediction"></a>

## Avaliação: Image prediction

In [23]:
df_prediction.describe()

,tweet_id,img_num,p1_conf,p2_conf,p3_conf
count,2.075000e+03,2075.000000,2075.000000,2.075000e+03,2.075000e+03
mean,7.384514e+17,1.203855,0.594548,1.345886e-01,6.032417e-02
std,6.785203e+16,0.561875,0.271174,1.006657e-01,5.090593e-02
min,6.660209e+17,1.000000,0.044333,1.011300e-08,1.740170e-10
25%,6.764835e+17,1.000000,0.364412,5.388625e-02,1.622240e-02
50%,7.119988e+17,1.000000,0.588230,1.181810e-01,4.944380e-02
75%,7.932034e+17,1.000000,0.843855,1.955655e-01,9.180755e-02
max,8.924206e+17,4.000000,1.000000,4.880140e-01,2.734190e-01


<a id="image-prediction:info"></info>

In [24]:
df_prediction.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null int64
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


In [25]:
df_prediction.sample(10)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
1595,798925684722855936,https://pbs.twimg.com/media/CxZaqh_WQAA7lY3.jpg,1,West_Highland_white_terrier,0.539463,True,cairn,0.184897,True,Norfolk_terrier,0.163024,True
1538,790723298204217344,https://pbs.twimg.com/media/CvaYgDOWgAEfjls.jpg,1,tub,0.479477,False,bathtub,0.325106,False,golden_retriever,0.078530,True
1224,744971049620602880,https://pbs.twimg.com/media/ClarNU8VAAEDrDt.jpg,1,toy_poodle,0.497755,True,golden_retriever,0.282017,True,miniature_poodle,0.090032,True
1270,749981277374128128,https://pbs.twimg.com/media/CmgBZ7kWcAAlzFD.jpg,1,bow_tie,0.533941,False,sunglasses,0.080822,False,sunglass,0.050776,False
505,675891555769696257,https://pbs.twimg.com/media/CWE_x33UwAEE3no.jpg,1,Italian_greyhound,0.305637,True,whippet,0.232057,True,Great_Dane,0.117806,True
2021,880935762899988482,https://pbs.twimg.com/media/DDm2Z5aXUAEDS2u.jpg,1,street_sign,0.251801,False,umbrella,0.115123,False,traffic_light,0.069534,False
603,679844490799091713,https://pbs.twimg.com/media/CW9K9VeVAAE0j-x.jpg,1,Airedale,0.903832,True,Border_terrier,0.034713,True,toy_poodle,0.021378,True
1327,757393109802180609,https://pbs.twimg.com/media/CoLNAq6WAAAkmdJ.jpg,2,Labrador_retriever,0.787125,True,Chesapeake_Bay_retriever,0.112676,True,Rottweiler,0.048039,True
1675,813157409116065792,https://pbs.twimg.com/media/C0jqVVOXUAAGJ0G.jpg,2,Siamese_cat,0.843911,False,Pembroke,0.070567,True,Cardigan,0.041916,True
1932,859196978902773760,https://pbs.twimg.com/ext_tw_video_thumb/85919...,1,Angora,0.224218,False,malamute,0.216163,True,Persian_cat,0.128383,False


<a id="image-prediction:p-values"></a>

Valores não padronizados para as variáveis `p1`, `p2` e `p3`

In [26]:
# Visualização de exemplos de valores nas colunas `p1`, `p2` e `p3`
df_prediction.loc[:, ['p1', 'p2', 'p3']].sample(5)

,p1,p2,p3
1707,curly-coated_retriever,flat-coated_retriever,Irish_water_spaniel
1735,golden_retriever,Labrador_retriever,Saluki
1379,Bernese_mountain_dog,Appenzeller,EntleBucher
1363,Norwegian_elkhound,malinois,German_shepherd
1972,Chihuahua,toy_terrier,muzzle


<a id="image-prediction:duplicated-urls"></a>

Verificar a duplicidade de Tweet ID (`tweet_id`) e Imagens (`jpg_url`)

In [27]:
# Verificar a duplicidade de IDs
df_prediction.tweet_id.nunique()

2075

Verificar a duplicidade na coluna `jpg_url`

In [28]:
df_prediction.jpg_url.duplicated().value_counts()

False    2009
True       66
Name: jpg_url, dtype: int64

Há 66 imagens duplicadas

In [29]:
# Validar indicadores de confiabiliade da predição
df_prediction.loc[:, ['p1_conf', 'p2_conf', 'p3_conf']].max()

p1_conf    1.000000
p2_conf    0.488014
p3_conf    0.273419
dtype: float64

------
<a id="avaliacao:twitter-api"></a>

## Avaliação: Twitter API Requests

In [30]:
df_tweets_api.describe()

,id,retweet_count,favorite_count
count,1.820000e+03,1820.000000,1820.000000
mean,7.237270e+17,2512.144505,6820.872527
std,5.777841e+16,4891.076169,11900.984651
min,6.660209e+17,11.000000,0.000000
25%,6.747671e+17,536.000000,1363.500000
50%,7.008223e+17,1118.000000,3153.000000
75%,7.617466e+17,2526.250000,7463.250000
max,8.924206e+17,82905.000000,163034.000000


<a id="tweets-api:info"></a>

In [31]:
df_tweets_api.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1820 entries, 0 to 1819
Data columns (total 10 columns):
id                       1820 non-null int64
created_at               1820 non-null object
in_reply_to_status_id    1820 non-null object
in_reply_to_user_id      1820 non-null object
is_quote_status          1820 non-null bool
retweet_count            1820 non-null int64
favorite_count           1820 non-null int64
media_url                1820 non-null object
retweeted                1820 non-null bool
favorited                1820 non-null bool
dtypes: bool(3), int64(3), object(4)
memory usage: 104.9+ KB


Os nomes das colunas `id` e `media_url` estão fora do padrão dos demais dataframes, respectivamente `tweet_id` e `jpg_url` 

In [32]:
df_tweets_api.sample(5)

,id,created_at,in_reply_to_status_id,in_reply_to_user_id,is_quote_status,retweet_count,favorite_count,media_url,retweeted,favorited
454,761750502866649088,Sat Aug 06 02:27:27 +0000 2016,None,None,False,4219,0,http://pbs.twimg.com/media/CYLDikFWEAAIy1y.jpg,False,False
1266,677573743309385728,Thu Dec 17 19:39:03 +0000 2015,None,None,False,762,2208,http://pbs.twimg.com/media/CWc5uR9WoAES8WK.jpg,False,False
489,757729163776290825,Tue Jul 26 00:08:05 +0000 2016,None,None,False,8677,0,http://pbs.twimg.com/media/CWyD2G_UEAAI9aa.jpg,False,False
798,710283270106132480,Thu Mar 17 01:55:02 +0000 2016,None,None,False,542,2218,http://pbs.twimg.com/media/Cdtu3WSUMAAFozO.jpg,False,False
1061,688804835492233216,Sun Jan 17 19:27:24 +0000 2016,None,None,False,205,1005,http://pbs.twimg.com/media/CY8gWE-WsAA60ZG.jpg,False,False


* `created_at` como `string`
* `in_reply_to_status_id` e `in_reply_to_userid` como `string`

<a id="tweets-api:duplicated"></a>
Verificar a duplicidade de valores

In [33]:
# Verificar se há IDs duplicados
df_tweets_api.id.duplicated().value_counts()

False    1820
Name: id, dtype: int64

In [34]:
# Verificar se há imagens duplicadas
df_tweets_api.media_url.duplicated().value_counts()

False    1759
True       61
Name: media_url, dtype: int64

Há tweets referenciando a mesma imagem

------

<a id="avaliacao:anotacoes"></a>

## Notas da avaliação
### Qualidade

#### `df_twitter_arc` - Twitter archive
1. [Base incompleta, contém apenas **2356** registros dos **5000** anunciados](#twitter-arc:info)
* [Coluna `source` sem valor à unidade de obsevação](#twitter-arc:source)
* [Nomes (`name`) preenchidos com valor literal igual a `'None'`, vazio](#twitter-arc:name)
* [Coluna `expanded_urls` sem valor à unidade de observação](#twitter-arc:expanded_urls)
* [Categorias \[`doggo`, `floofer`, `pupper`, `puppo`\] com literal `'None'`](#twitter-arc:category)


#### `df_prediction` - Image prediction
1. [Previsões \[`p1`, `p2`, `p3`\] com nomes não padronizados](#image-prediction:p-values)
* [Duplicidade na coluna `jgp_url`](#image-prediction:duplicated-urls)


#### `df_tweets_api` - Twitter API
1. [Diferentes Tweets referenciando a mesma imagem, duplicidade](#tweets-api:duplicated)


### Organização

#### `df_twitter_arc` - Twitter archive
1. [As colunas `in_reply_to_status_id`, `in_reply_to_user_id`, `retweeted_status_id`, `retweeted_status_user_id` estão em `float64`](#twitter-arc:info)
* [As colunas `timestamp` e `retweet_status_timestamp` estão em `string`](#twitter-arc:info)
* [Uma variável em várias colunas, `doggo`, `floofer`, `pupper` e `puppo`](#twitter-arc:category)


#### `df_prediction` - Image prediction
1. [Colunas como variáveis na identificação das predições](#image-prediction:info)


#### `df_tweets_api` - Twitter API
1. [Os nomes das colunas `id` e `media_url` estão fora do padrão dos demais dataframes, respectivamente `tweet_id` e `jpg_url` ](#tweets-api:info)
* [Data de criação (`created_at`) como `string`](#tweets-api:info)
* [As colunas `in_reply_to_status_id` e `in_reply_to_user_id` em `string`](#tweets-api:info)

------

<a id="limpeza"></a>

# Limpeza

<a id="limpeza:definicao"></a>

## Limpeza: Definição 

#### `df_twitter_arc` - Twitter archive
1. [Base incompleta, de **5000**, apenas **2356** estão disponíveis](#limpeza:twitter-arc:completeness)
    * OK - Nada a ser feito
* [Coluna `source` sem valor à unidade de observação](#limpeza:twitter-arc:source)
    * O conteúdo apresenta um link para download do App Twitter, portanto não agrega valor à unidade de observação, a coluna pode ser removida
* [Nomes (`name`) não preenchidos com valor literal igual a `'None'`](#limpeza:twitter-arc:name)
    * Alterar valores como 'None' para `np.nan`
* [Coluna `expanded_urls` sem valor à unidade de observação](#limpeza:twitter-arc:expanded_urls)(#twitter-arc:expanded_urls)
    * Não apresentam valor à unidade de observação, são URLs para os tweets em questão. Remover a coluna `expanded_urls`.
* [Categorias \[doggo, floofer, pupper, puppo\] com literal `'None'`](#limpeza:twitter-arc:category)
    * Aplicar valor `np.nan` para nomes iguais a 'None'   
* [As colunas `in_reply_to_status_id`, `in_reply_to_user_id`, `retweeted_status_id`, `retweeted_status_user_id` estão em `float64`](#limpeza:twitter-arc:info)
    * Linhas com valor para as colunas `retweeted_status_id` ou `retweeted_status_user_id` podem ser descartadas por indicarem retweet, o objetivo é analisar somente tweets originais
    * Após a remoção retweets, as colunas `retweeted_status_id`, `retweeted_status_user_id`, `retweeted_status_timestamp`, podem ser descartadas
    * As colunas `in_reply_to_status_id` e `in_reply_to_user_id` serão retratadas no dataframe `df_tweets_api`
* [As colunas `timestamp` e `retweet_status_timestamp` estão em `string` e deveriam ser do tipo `datetime`](#limpeza:twitter-arc:datetime)
    * Converter valores para `datetime`
* [Uma variável em várias colunas, `doggo`, `floofer`, `pupper` e `puppo`](#limpeza:twitter-arc:category-merge)
    * Cada variável em apenas uma coluna,  `category`


#### `df_prediction` - Image prediction
1. [Previsões \[`p1`, `p2`, `p3`\] com nomes não padronizados](#limpeza:image-prediction:names)
    * Converter os valores para 'lower case' e substituir '_' por espaços
* [Previsões repetidas sobre a mesma imagem](#limpeza:image-prediction:duplicated-urls)
    * Descartar predições sobre imagens repetidas
* [Colunas como variáveis na identificação das predições](#limpeza:image-prediction:info)
    * Transpor as variáveis nas colunas para linhas


#### `df_tweets_api` - Twitter API
1. [Os nomes das colunas `id` e `media_url` estão fora do padrão dos demais dataframes, respectivamente `tweet_id` e `jpg_url` ](#limpeza:tweets-api:columns)
* [Diferentes Tweets referenciando a mesma imagem, duplicidade](#limpeza:tweets-api:duplicated)
* [Data de criação (`created_at`) como `string`](#limpeza:tweets-api:datetime)
    * Converter datas em string para o formato `datetime`
* [Valor do id na coluna `in_reply_to_status_id` e `in_reply_to_user_id` como `string`](#limpeza:tweets-api:info)
    * Converter para Inteiro as colunas `in_reply_to_status_id` e `in_reply_to_user_id` e atribuir np.nan as que estiverem sem valor

<a id="limpeza:twitter-archive"></a>

## Limpeza: Twitter archive

In [35]:
# Criar cópia do dataframe para a limpeza
df_arch_clean = df_twitter_arc.copy()

<a id="limpeza:twitter-arc:completeness"></a>

### 1. [Base incompleta, de 5000, apenas 2356 estão disponíveis](#limpeza:twitter-arc:completeness)
Nada a ser feito

In [36]:
df_arch_clean.shape

(2356, 17)

<a id="limpeza:twitter-arc:source"></a>

### 2. [Coluna `source` sem valor à unidade de observação](#limpeza:twitter-arc:source)
* O conteúdo apresenta um link para download do App Twitter, portanto não agrega valor à unidade de observação, a coluna pode ser removida

In [37]:
# Remoção da coluna `source`
df_arch_clean.drop('source', axis='columns', inplace=True)

#### Teste: Remoção coluna `source`

In [38]:
# Conferir que a coluna `source` foi removida
df_arch_clean.columns

Index(['tweet_id', 'in_reply_to_status_id', 'in_reply_to_user_id', 'timestamp',
       'text', 'retweeted_status_id', 'retweeted_status_user_id',
       'retweeted_status_timestamp', 'expanded_urls', 'rating_numerator',
       'rating_denominator', 'name', 'doggo', 'floofer', 'pupper', 'puppo'],
      dtype='object')

<a id="limpeza:twitter-arc:name"></a>

### 3. [Nomes (`name`) não preenchidos com valor literal igual a `'None'`](#limpeza:twitter-arc:name)
    * Alterar valores como 'None' para `np.nan`

In [39]:
# Contagem de nomes iguais a 'None'
df_arch_clean[df_arch_clean.name.str.lower() == 'none'].name.count()

745

In [40]:
# Função para substituição de valores literais 'None' por np.nan
def set_nan_at_none(value):
    if str(value).lower() == 'none':
        return np.nan
    else:
        return value

In [41]:
# Aplicar `np.nan` aos nomes iguais a 'None'
df_arch_clean.name = df_arch_clean.name.apply(set_nan_at_none)

#### Teste: Aplicação de `np.nan` aos nomes como 'None'

In [42]:
df_arch_clean.name.isnull().sum()

745

In [43]:
df_arch_clean[df_arch_clean.name.str.lower() == 'none'].name.count()

0

In [44]:
df_arch_clean.name.sample(10)

423     Duchess
2001      light
77          Aja
439      Oliver
467         NaN
1546    Chipson
1561      Louis
1351          a
1477        NaN
1423        NaN
Name: name, dtype: object

<a id="limpeza:twitter-arc:expanded_urls"></a>

### 4. [Coluna `expanded_urls` sem valor à unidade de observação](#limpeza:twitter-arc:expanded_urls)
* Não apresentam valor à unidade de observação, são URLs para os tweets em questão. Remover a coluna `expanded_urls`

In [45]:
df_arch_clean.drop('expanded_urls', axis='columns', inplace=True)

#### Teste: Remoção da coluna `expanded_urls`

In [46]:
df_arch_clean.columns

Index(['tweet_id', 'in_reply_to_status_id', 'in_reply_to_user_id', 'timestamp',
       'text', 'retweeted_status_id', 'retweeted_status_user_id',
       'retweeted_status_timestamp', 'rating_numerator', 'rating_denominator',
       'name', 'doggo', 'floofer', 'pupper', 'puppo'],
      dtype='object')

<a id="limpeza:twitter-arc:category"></a>

### 5. [Categorias \[`doggo`, `floofer`, `pupper`, `puppo`\] com literal `'None'`](#limpeza:twitter-arc:category)
* Aplicar valor `np.nan` para nomes iguais a 'None'   

In [47]:
# Trocar valores 'None' para np.nan nas colunas `doggo`, `floofer`, `pupper`, `puppo`
df_arch_clean.doggo = df_arch_clean.doggo.apply(set_nan_at_none)
df_arch_clean.doggo.isna().value_counts()

True     2259
False      97
Name: doggo, dtype: int64

In [48]:
df_arch_clean.floofer = df_arch_clean.floofer.apply(set_nan_at_none)
df_arch_clean.floofer.isna().value_counts()

True     2346
False      10
Name: floofer, dtype: int64

In [49]:
df_arch_clean.pupper = df_arch_clean.pupper.apply(set_nan_at_none)
df_arch_clean.pupper.isna().value_counts()

True     2099
False     257
Name: pupper, dtype: int64

In [50]:
df_arch_clean.puppo = df_arch_clean.puppo.apply(set_nan_at_none)
df_arch_clean.puppo.isna().value_counts()

True     2326
False      30
Name: puppo, dtype: int64

In [51]:
# Soma de valores preenchidos nas classificações
sum_categories = 0
sum_categories += df_arch_clean.doggo.notna().sum()
sum_categories += df_arch_clean.floofer.notna().sum()
sum_categories += df_arch_clean.pupper.notna().sum()
sum_categories += df_arch_clean.puppo.notna().sum()

sum_categories

394

Apenas 394 dos 2346 registros foram classificados

<a id="limpeza:twitter-arc:info"></a>

### 6. [As colunas `in_reply_to_status_id`, `in_reply_to_user_id`, `retweeted_status_id`, `retweeted_status_user_id` estão em `float64`](#limpeza:twitter-arc:info)
* Linhas com algum valor para as colunas `retweeted_status_id` ou `retweeted_status_user_id` podem ser descartadas por indicarem retweet, e o objetivo é analisar somente tweets originais
* Após a remoção retweets, as colunas `retweeted_status_id`, `retweeted_status_user_id`, `retweeted_status_timestamp`, podem ser descartadas
* As colunas `in_reply_to_status_id` e `in_reply_to_user_id` serão retratadas no dataframe `df_tweets_api`

#### Identificação dos tweets originais
df_arch_clean.loc[:,'retweeted_status_id'].isna().value_counts()

In [52]:
df_arch_clean.retweeted_status_id.size

2356

In [53]:
df_arch_clean.loc[:,'retweeted_status_id'].isna().value_counts()

True     2175
False     181
Name: retweeted_status_id, dtype: int64

Dos 2356 registros, 181 são retweets

In [54]:
# Remoção dos retweets
df_arch_clean.drop(df_arch_clean[df_arch_clean.retweeted_status_id.notna()].index, axis=0, inplace=True)

#### Teste: Remoção de Retweets

In [55]:
# Consultar a existência de colunas com retweet_status_preenchido
df_arch_clean.loc[:,'retweeted_status_id'].isna().value_counts()

True    2175
Name: retweeted_status_id, dtype: int64

Remoção das colunas referentes a retweets

In [56]:
df_arch_clean.drop(['retweeted_status_id', 'retweeted_status_user_id', 'retweeted_status_timestamp'], axis='columns', inplace=True)

#### Teste: Verificação da remoção das colunas referentes a retweets

In [57]:
df_arch_clean.columns

Index(['tweet_id', 'in_reply_to_status_id', 'in_reply_to_user_id', 'timestamp',
       'text', 'rating_numerator', 'rating_denominator', 'name', 'doggo',
       'floofer', 'pupper', 'puppo'],
      dtype='object')

<a id="limpeza:twitter-arc:datetime"></a>

### 7. [As colunas `timestamp` e `retweet_status_timestamp` estão em `string` e deveriam ser do tipo `datetime`](#limpeza:twitter-arc:datetime)
* Converter valores para `datetime`

Haja vista que a coluna `retweet_status_timestamp` foi removida no passo anterior, não será necessária a sua conversão

In [58]:
df_arch_clean.loc[:, ['timestamp']].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2175 entries, 0 to 2355
Data columns (total 1 columns):
timestamp    2175 non-null object
dtypes: object(1)
memory usage: 34.0+ KB


In [59]:
df_arch_clean.timestamp = pd.to_datetime(df_arch_clean.timestamp)

#### Teste: Veriificação da conversão de `string` para `datetime`

In [60]:
df_arch_clean.loc[:, ['timestamp']].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2175 entries, 0 to 2355
Data columns (total 1 columns):
timestamp    2175 non-null datetime64[ns, UTC]
dtypes: datetime64[ns, UTC](1)
memory usage: 34.0 KB


<a id="limpeza:twitter-arc:category-merge"></a>

### 8. [Uma variável em várias colunas, `doggo`, `floofer`, `pupper` e `puppo`](#limpeza:twitter-arc:category-merge)
* Cada variável em apenas uma coluna, reduzi-las a coluna `category`

In [61]:
df_arch_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2175 entries, 0 to 2355
Data columns (total 12 columns):
tweet_id                 2175 non-null int64
in_reply_to_status_id    78 non-null float64
in_reply_to_user_id      78 non-null float64
timestamp                2175 non-null datetime64[ns, UTC]
text                     2175 non-null object
rating_numerator         2175 non-null int64
rating_denominator       2175 non-null int64
name                     1495 non-null object
doggo                    87 non-null object
floofer                  10 non-null object
pupper                   234 non-null object
puppo                    25 non-null object
dtypes: datetime64[ns, UTC](1), float64(2), int64(3), object(6)
memory usage: 220.9+ KB


In [62]:
df_arch_clean.loc[:, 'doggo':].notna().sum()

doggo       87
floofer     10
pupper     234
puppo       25
dtype: int64

In [63]:
df_arch_clean.loc[:, 'doggo':].notna().sum().sum()

356

De **2175** registros, apenas **356** estão classificados

Criar uma lista das categorias para mesclar as colunas, considerando a coluna a esquerda como prioritaria sobre valores nulos a direita.

In [64]:
category = df_arch_clean.doggo
category = category.combine_first(df_arch_clean.floofer)
category = category.combine_first(df_arch_clean.pupper)
category = category.combine_first(df_arch_clean.puppo)

df_arch_clean['category'] = category

In [65]:
df_arch_clean.category.notna().sum()

344

Após a unificação das colunas de classificação, pode-se ver que resultou em apenas **344** registros, diferente dos **356** iniciais. O que indica que haviam registros com mais de uma classificação.

In [66]:
df_arch_clean.drop(['doggo', 'floofer', 'pupper', 'puppo'], axis='columns', inplace=True)

#### Teste: Verificar a criação da coluna `category` a remoção das colunas `doggo`, `floofer`, `pupper` e `puppo`. 

In [67]:
df_arch_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2175 entries, 0 to 2355
Data columns (total 9 columns):
tweet_id                 2175 non-null int64
in_reply_to_status_id    78 non-null float64
in_reply_to_user_id      78 non-null float64
timestamp                2175 non-null datetime64[ns, UTC]
text                     2175 non-null object
rating_numerator         2175 non-null int64
rating_denominator       2175 non-null int64
name                     1495 non-null object
category                 344 non-null object
dtypes: datetime64[ns, UTC](1), float64(2), int64(3), object(3)
memory usage: 169.9+ KB


------
<a id="limpeza:image-prediction"></a>

## Limpeza: Image prediction 

In [68]:
# Cópia do dataframe para limpeza
df_predict_clean = df_prediction.copy()

<a id="limpeza:image-prediction:names"></a>

### 1. [Previsões \[`p1`, `p2`, `p3`\] com nomes não padronizados](#limpeza:image-prediction:names)
* Converter os valores para 'lower case' e substituir '_' por espaços

In [69]:
clean_prediction_name = lambda value: str(value).lower().replace('_', ' ')

In [70]:
df_predict_clean.loc[:, ['p1', 'p2', 'p3']].sample(5)

,p1,p2,p3
629,Pomeranian,golden_retriever,Chihuahua
1657,flat-coated_retriever,Labrador_retriever,curly-coated_retriever
1567,pedestal,fountain,brass
1937,lakeside,dock,canoe
791,bath_towel,radiator,Maltese_dog


Aplicar a padronização dos nomes em cada uma das colunas de predição

In [71]:
df_predict_clean.p1 = df_predict_clean.p1.apply(clean_prediction_name)

In [72]:
df_predict_clean.p2 = df_predict_clean.p2.apply(clean_prediction_name)

In [73]:
df_predict_clean.p3 = df_predict_clean.p3.apply(clean_prediction_name)

#### Teste: Verificar padronização dos nomes das predições

In [74]:
df_predict_clean.loc[:, ['p1', 'p2', 'p3']].sample(5)

,p1,p2,p3
1237,boston bull,guinea pig,cardigan
965,papillon,pekinese,saint bernard
410,brabancon griffon,chihuahua,golden retriever
170,lynx,tabby,koala
750,pomeranian,pekinese,keeshond


<a id="limpeza:image-prediction:duplicated-urls"></a>

### 2. [Predições repetidas sobre a mesma imagem](#limpeza:image-prediction:duplicated-urls)
* Descartar predições sobre imagens repetidas

In [75]:
df_predict_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null int64
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


In [76]:
df_predict_clean.loc[:, ['jpg_url', 'p1', 'p2', 'p3']].duplicated().value_counts()

False    2009
True       66
dtype: int64

Dos **2075** registros, **66** são sobre a mesma imagem. Entretanto, seguindo o objetivo de trabalhar com apenas Tweets originais, além de descartar duplicidade é interessante também descartar as refêrencias a Retweets. Para isso, será feita uma consulta a seleção de Tweets originais, que foi efetuada no passo [6 da limpeza do Twitter Archive](#limpeza:twitter-arc:info)

In [77]:
df_predict_clean = df_predict_clean.loc[df_predict_clean.tweet_id.isin(df_arch_clean.tweet_id)]

#### Teste: Seleção de Tweets originais

In [78]:
df_predict_clean.tweet_id.isin(df_arch_clean.tweet_id).value_counts()

True    1994
Name: tweet_id, dtype: int64

Apenas dos **2075**, **1994** são de Tweets originais

------

Dos Tweets originai ainda é necessário verificar a existência de duplicidade de predições.

In [79]:
df_predict_clean.loc[:, ['jpg_url']].duplicated().value_counts()

False    1994
dtype: int64

Nenhuma duplicidade foi encontrada após a remoção das referências de Retweets

In [80]:
df_predict_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1994 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    1994 non-null int64
jpg_url     1994 non-null object
img_num     1994 non-null int64
p1          1994 non-null object
p1_conf     1994 non-null float64
p1_dog      1994 non-null bool
p2          1994 non-null object
p2_conf     1994 non-null float64
p2_dog      1994 non-null bool
p3          1994 non-null object
p3_conf     1994 non-null float64
p3_dog      1994 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 161.6+ KB


<a id="limpeza:image-prediction:info"></a>

### 3. [Colunas como variáveis na identificação das predições](#limpeza:image-prediction:info)
* 3.1. Transpor as variáveis nas colunas para linhas:
    * 3.1.1. [`[p1, p2, p3]`, para `prediction_try` e seus valores para `prediction_breed`](#limpeza:image-prediction:try)
    * 3.1.2. [`[p1_conf, p2_conf, p3_conf]`, para `confidence`](#limpeza:image-prediction:conf)
    * 3.1.3. [`[p1_dog, p2_dog, p3_dog]`, para `is_a_dog`](#limpeza:image-prediction:dog)
* 3.2. [Unir os dataframes de prediction](#limpeza:image-prediction:merge)

In [81]:
df_predict_clean.sample(5)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
865,697463031882764288,https://pbs.twimg.com/media/Ca3i7CzXIAMLhg8.jpg,1,labrador retriever,0.999885,True,golden retriever,0.000098,True,pug,0.000008,True
2025,881906580714921986,https://pbs.twimg.com/media/DD0pWm9XcAAeSBL.jpg,1,weimaraner,0.291539,True,chesapeake bay retriever,0.278966,True,koala,0.127017,False
2007,877736472329191424,https://pbs.twimg.com/media/DC5YqoQW0AArOLH.jpg,2,chesapeake bay retriever,0.837956,True,labrador retriever,0.062034,True,weimaraner,0.040599,True
305,671520732782923777,https://pbs.twimg.com/media/CVG4i9UWEAAUH3U.jpg,1,pomeranian,0.551031,True,pekinese,0.135262,True,gibbon,0.061557,False
1376,763183847194451968,https://pbs.twimg.com/media/CpdfpzKWYAAWSUi.jpg,1,miniature poodle,0.354674,True,toy poodle,0.338642,True,teddy,0.155828,False


<a id="limpeza:image-prediction:try"></a>

#### 3.1.1. [Organização das colunas `[p1, p2, p3]`, para `prediction_try` e seus valores para `prediction_breed` ](#limpeza:image-prediction:try)

In [82]:
# Organizando as variáveis de tentativa de predição e seu palpite
df_predict_try = pd.melt(df_predict_clean,
        id_vars=['tweet_id', 'jpg_url'],
        value_vars=['p1', 'p2', 'p3'],
        var_name=['prediction_try'],
        value_name='prediction_breed'
       )

In [83]:
df_predict_try.sample(5)

,tweet_id,jpg_url,prediction_try,prediction_breed
2346,672591762242805761,https://pbs.twimg.com/media/CVWGotpXAAMRfGq.jpg,p2,great pyrenees
1159,734912297295085568,https://pbs.twimg.com/media/CjLuzPvUoAAbU5k.jpg,p1,maltese dog
3932,880095782870896641,https://pbs.twimg.com/media/DDa6ckbXgAAM1vV.jpg,p2,rhodesian ridgeback
2876,698953797952008193,https://pbs.twimg.com/media/CbMuxV5WEAAIBjy.jpg,p2,redbone
4823,694329668942569472,https://pbs.twimg.com/media/CaLBJmOWYAQt44t.jpg,p3,saint bernard


Conversão dos valores da tentativa para número inteiro

In [84]:
# Metodo para retirar o índice da tentativa dos nomes das colunas
get_prediction_index = lambda value: int(re.sub(r'[^\d]', '', str(value)))

In [85]:
df_predict_try.prediction_try = df_predict_try.prediction_try.apply(get_prediction_index)

In [86]:
df_predict_try.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5982 entries, 0 to 5981
Data columns (total 4 columns):
tweet_id            5982 non-null int64
jpg_url             5982 non-null object
prediction_try      5982 non-null int64
prediction_breed    5982 non-null object
dtypes: int64(2), object(2)
memory usage: 187.0+ KB


In [87]:
df_predict_try.sample(5)

,tweet_id,jpg_url,prediction_try,prediction_breed
68,667182792070062081,https://pbs.twimg.com/media/CUJPNjOWsAAZRqP.jpg,1,golden retriever
196,669625907762618368,https://pbs.twimg.com/media/CUr9NjgU8AEpf5w.jpg,1,seat belt
4105,668171859951755264,https://pbs.twimg.com/media/CUXSwy8W4AA6uet.jpg,3,giant panda
1269,750011400160841729,https://pbs.twimg.com/media/CmfmvGUWgAAuVKD.jpg,1,muzzle
5664,821522889702862852,https://pbs.twimg.com/media/C2aitIUXAAAG-Wi.jpg,3,miniature pinscher


<a id="limpeza:image-prediction:conf"></a>

#### 3.1.2. [`[p1_conf, p2_conf, p3_conf]`, para `confidence`](#limpeza:image-prediction:conf)

In [88]:
# Movendo o nível de confiança das predições para uma coluna apenas
df_predict_conf = pd.melt(df_predict_clean,
                         id_vars=['tweet_id', 'jpg_url'],
                         value_vars=['p1_conf', 'p2_conf', 'p3_conf'],
                         var_name=['prediction_try'],
                         value_name='confidence'
                        )

In [89]:
df_predict_conf.sample(5)

,tweet_id,jpg_url,prediction_try,confidence
5184,740711788199743490,https://pbs.twimg.com/media/CkeJcNkXEAAcrks.jpg,p3_conf,0.049656
3980,890729181411237888,https://pbs.twimg.com/media/DFyBahAVwAAhUTd.jpg,p2_conf,0.178406
2198,669926384437997569,https://pbs.twimg.com/media/CUwOfnDWcAIXryP.jpg,p2_conf,0.010231
4407,674045139690631169,https://pbs.twimg.com/media/CVqwedgXIAEAT6A.jpg,p3_conf,0.043178
882,698953797952008193,https://pbs.twimg.com/media/CbMuxV5WEAAIBjy.jpg,p1_conf,0.382378


In [90]:
# Atribuir apenas valores inteiros aos indicadores de tentativas
df_predict_conf.prediction_try = df_predict_conf.prediction_try.apply(get_prediction_index)

In [91]:
df_predict_conf.sample(5)

,tweet_id,jpg_url,prediction_try,confidence
3355,762464539388485633,https://pbs.twimg.com/media/CpTRc4DUEAAYTq6.jpg,2,0.000023
326,672068090318987265,https://pbs.twimg.com/media/CVOqW8eUkAESTHj.jpg,1,0.863385
2296,671518598289059840,https://pbs.twimg.com/media/CVG2l9jUYAAwg-w.jpg,2,0.111472
5968,889638837579907072,https://pbs.twimg.com/media/DFihzFfXsAYGDPR.jpg,3,0.001498
3528,793500921481273345,https://pbs.twimg.com/media/CwMU34YWIAAz1nU.jpg,2,0.219904


In [92]:
df_predict_conf.shape

(5982, 4)

<a id="limpeza:image-prediction:dog"></a>

#### 3.1.3. [`[p1_dog, p2_dog, p3_dog]`, para `is_a_dog`](#limpeza:image-prediction:dog)

In [93]:
# Movendo o resultado da predição, `p#_dog` para `is_a_dog`
df_predict_dog = pd.melt(df_predict_clean,
                        id_vars=['tweet_id', 'jpg_url'],
                        value_vars=['p1_dog', 'p2_dog', 'p3_dog'],
                        var_name=['prediction_try'],
                        value_name='is_a_dog')

In [94]:
df_predict_dog.sample(5)

,tweet_id,jpg_url,prediction_try,is_a_dog
70,667192066997374976,https://pbs.twimg.com/media/CUJXpRBXIAAN0yz.jpg,p1_dog,True
4377,673576835670777856,https://pbs.twimg.com/media/CVkGjsxU8AA5OYX.jpg,p3_dog,False
239,670444955656130560,https://pbs.twimg.com/media/CU3mITUWIAAfyQS.jpg,p1_dog,True
4184,669625907762618368,https://pbs.twimg.com/media/CUr9NjgU8AEpf5w.jpg,p3_dog,True
3231,746906459439529985,https://pbs.twimg.com/media/Cl2LdofXEAATl7x.jpg,p2_dog,False


In [95]:
# Atribuir apenas valores inteiros aos indicadores de tentativas
df_predict_dog.prediction_try = df_predict_dog.prediction_try.apply(get_prediction_index)

In [96]:
df_predict_dog.sample(5)

,tweet_id,jpg_url,prediction_try,is_a_dog
854,696754882863349760,https://pbs.twimg.com/media/Cate3eLUcAEIuph.jpg,1,False
2117,668248472370458624,https://pbs.twimg.com/media/CUYYcMfXAAAixe7.jpg,2,True
1613,812372279581671427,https://pbs.twimg.com/media/C0YgO3DW8AAz98O.jpg,1,True
3254,749036806121881602,https://pbs.twimg.com/media/CmUciKgWIAA97sH.jpg,2,True
2676,683852578183077888,https://pbs.twimg.com/media/CX2ISqSWYAAEtCF.jpg,2,False


In [97]:
df_predict_dog.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5982 entries, 0 to 5981
Data columns (total 4 columns):
tweet_id          5982 non-null int64
jpg_url           5982 non-null object
prediction_try    5982 non-null int64
is_a_dog          5982 non-null bool
dtypes: bool(1), int64(2), object(1)
memory usage: 146.1+ KB


<a id="limpeza:image-prediction:merge"></a>

#### 3.2. [Unir os dataframes de prediction](#limpeza:image-prediction:merge)

In [98]:
# Merge
df_predict_clean = pd.merge(df_predict_try, df_predict_conf, how='left', on=['tweet_id', 'jpg_url', 'prediction_try'])

In [99]:
df_predict_clean.shape

(5982, 5)

In [100]:
df_predict_clean.sample(5)

,tweet_id,jpg_url,prediction_try,prediction_breed,confidence
3083,719551379208073216,https://pbs.twimg.com/media/CfxcKU6W8AE-wEx.jpg,2,siberian husky,0.076435
4597,680115823365742593,https://pbs.twimg.com/media/CXBBurSWMAELewi.jpg,3,boston bull,0.000028
4999,709556954897764353,https://pbs.twimg.com/media/CdjaSFCWAAAJZh3.jpg,3,labrador retriever,0.087051
3980,890729181411237888,https://pbs.twimg.com/media/DFyBahAVwAAhUTd.jpg,2,eskimo dog,0.178406
5579,806629075125202948,https://pbs.twimg.com/media/CzG425nWgAAnP7P.jpg,3,cocker spaniel,0.046403


In [101]:
df_predict_clean = pd.merge(df_predict_clean, df_predict_dog, how='left', on=['tweet_id', 'jpg_url', 'prediction_try'])

In [102]:
df_predict_clean.shape

(5982, 6)

**Teste: Verificar Merge dos dataframes de predição**

In [103]:
df_predict_clean.sample(5)

,tweet_id,jpg_url,prediction_try,prediction_breed,confidence,is_a_dog
655,682303737705140231,https://pbs.twimg.com/media/CXgHoLnWAAA8i52.jpg,1,seat belt,0.997659,False
3704,827933404142436356,https://pbs.twimg.com/media/C31pCN4UcAAOLNH.jpg,2,tibetan mastiff,0.104831,True
4966,707315916783140866,https://pbs.twimg.com/media/CdDkEkHWwAAAeUJ.jpg,3,appenzeller,0.003971,True
3033,712438159032893441,https://pbs.twimg.com/media/CeMWubMWwAA6GwF.jpg,2,great pyrenees,0.069457,True
5111,727524757080539137,https://pbs.twimg.com/media/Chiv6BAW4AAiQvH.jpg,3,chow,0.003941,True


------
<a id="limpeza:twitter-api"></a>

## Limpeza: Twitter API

In [104]:
# Cópia do dataframe para limpeza
df_api_clean = df_tweets_api.copy()

In [105]:

df_api_clean.sample(3)

,id,created_at,in_reply_to_status_id,in_reply_to_user_id,is_quote_status,retweet_count,favorite_count,media_url,retweeted,favorited
1197,680583894916304897,Sat Dec 26 03:00:19 +0000 2015,None,None,False,1427,3769,http://pbs.twimg.com/media/CXHrcFYWcAEE5_L.jpg,False,False
181,813217897535406080,Mon Dec 26 03:00:30 +0000 2016,None,None,False,7928,19774,http://pbs.twimg.com/media/C0khWkVXEAI389B.jpg,False,False
1237,678798276842360832,Mon Dec 21 04:44:55 +0000 2015,None,None,False,1253,3595,http://pbs.twimg.com/media/CWuTbAKUsAAvZHh.jpg,False,False


<a id="limpeza:tweets-api:columns"></a>

### 1. [Os nomes das colunas `id` e `media_url` estão fora do padrão dos demais dataframes, respectivamente `tweet_id` e `jpg_url` ](#limpeza:tweets-api:columns)

In [106]:
df_api_clean.rename(columns={'id':'tweet_id', 'media_url': 'jpg_url'}, inplace=True)

**Teste: Verificar colunas renomeadas**

In [107]:
df_api_clean.columns

Index(['tweet_id', 'created_at', 'in_reply_to_status_id',
       'in_reply_to_user_id', 'is_quote_status', 'retweet_count',
       'favorite_count', 'jpg_url', 'retweeted', 'favorited'],
      dtype='object')

<a id="limpeza:tweets-api:duplicated"></a>

### 2. [Diferentes Tweets referenciando a mesma imagem, duplicidade](#limpeza:tweets-api:duplicated)

Assim como na [limpeza de duplicidade de imagens no dataframe de predições](#limpeza:image-prediction:duplicated-urls), é importante de remover os tweets não originais, os retweets. Com auxílio do dataframe `df_arch_clean`, que possui somente referências a tweets originais, será possível identifica-los e descartar os tweets indesejados.

In [108]:
df_api_clean.shape

(1820, 10)

In [109]:
df_api_clean = df_api_clean.loc[df_api_clean.tweet_id.isin(df_arch_clean.tweet_id)]

In [110]:
df_api_clean.shape

(1746, 10)

Ao remover os ids que referenciavam a **retweets**, houve uma redução de **74** registros, de **1820** para **1746**

<a id="limpeza:tweets-api:datetime"></a>

### 3. [Data de criação (`created_at`) como `string`](#limpeza:tweets-api:datetime)
* Converter datas em string para o formato `datetime`

In [111]:
df_api_clean.created_at = pd.to_datetime(df_api_clean.created_at)

**Teste: Validar tipo de variável em `created_at`**

In [112]:
df_api_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1746 entries, 0 to 1819
Data columns (total 10 columns):
tweet_id                 1746 non-null int64
created_at               1746 non-null datetime64[ns, UTC]
in_reply_to_status_id    1746 non-null object
in_reply_to_user_id      1746 non-null object
is_quote_status          1746 non-null bool
retweet_count            1746 non-null int64
favorite_count           1746 non-null int64
jpg_url                  1746 non-null object
retweeted                1746 non-null bool
favorited                1746 non-null bool
dtypes: bool(3), datetime64[ns, UTC](1), int64(3), object(3)
memory usage: 114.2+ KB


In [113]:
df_api_clean.sample(5)

,tweet_id,created_at,in_reply_to_status_id,in_reply_to_user_id,is_quote_status,retweet_count,favorite_count,jpg_url,retweeted,favorited
65,849051919805034497,2017-04-04 00:12:06+00:00,None,None,False,8870,43543,http://pbs.twimg.com/media/C8hwNxbXYAAwyVG.jpg,False,False
1421,673680198160809984,2015-12-07 01:47:30+00:00,None,None,False,479,1385,http://pbs.twimg.com/media/CVlkid8WoAAqDlB.jpg,False,False
1047,689599056876867584,2016-01-20 00:03:21+00:00,None,None,False,5851,12093,http://pbs.twimg.com/media/CZHyrvOXEAEin-A.jpg,False,False
1139,683828599284170753,2016-01-04 01:53:37+00:00,None,None,False,1121,2916,http://pbs.twimg.com/media/CX1ye7HUMAADDzh.jpg,False,False
954,697575480820686848,2016-02-11 00:18:49+00:00,None,None,False,524,2121,http://pbs.twimg.com/media/Ca5JMvMUsAAGMll.jpg,False,False


<a id="limpeza:tweets-api:info"></a>

### 4. [Valor do id na coluna `in_reply_to_status_id` e `in_reply_to_user_id` como `string`](#limpeza:tweets-api:info)
* Converter para Inteiro as colunas `in_reply_to_status_id` e `in_reply_to_user_id` e atribuir np.nan as que estiverem sem valor

In [114]:
df_api_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1746 entries, 0 to 1819
Data columns (total 10 columns):
tweet_id                 1746 non-null int64
created_at               1746 non-null datetime64[ns, UTC]
in_reply_to_status_id    1746 non-null object
in_reply_to_user_id      1746 non-null object
is_quote_status          1746 non-null bool
retweet_count            1746 non-null int64
favorite_count           1746 non-null int64
jpg_url                  1746 non-null object
retweeted                1746 non-null bool
favorited                1746 non-null bool
dtypes: bool(3), datetime64[ns, UTC](1), int64(3), object(3)
memory usage: 114.2+ KB


Converter os valores em `None` para np.nan em `in_reply_to_status_id` e `in_reply_to_user_id`

In [115]:
def set_zero_at_none(value):
    if str(value).lower() == 'none':
        return '0'
    else:
        return value

In [116]:
df_api_clean.in_reply_to_status_id = df_api_clean.in_reply_to_status_id.apply(set_zero_at_none)

In [117]:
df_api_clean.in_reply_to_user_id = df_api_clean.in_reply_to_user_id.apply(set_zero_at_none)

In [118]:
df_api_clean.loc[:, ['in_reply_to_user_id', 'in_reply_to_status_id']] = df_api_clean.loc[:, ['in_reply_to_user_id', 'in_reply_to_status_id']].astype('int64')

Listar exemplos com valores 'None'substituídos por '0' e convertidos para `int64`

In [119]:
df_api_clean.loc[:, ['in_reply_to_user_id', 'in_reply_to_status_id']].sample(5)

,in_reply_to_user_id,in_reply_to_status_id
39,0,0
838,0,0
534,0,0
1716,0,0
979,0,0


Converter valores iguais a 0 por `np.nan`

In [120]:
# Função para conversão de np.nan para 0
set_nan_at_zero = lambda value: np.nan if value is 0 else value

In [121]:
df_api_clean.in_reply_to_status_id = df_api_clean.in_reply_to_status_id.apply(set_nan_at_zero)

In [122]:
df_api_clean.in_reply_to_user_id = df_api_clean.in_reply_to_user_id.apply(set_nan_at_zero)

**Teste: Confirmar conversão para inteiro das colunas `in_reply_to_status_id` e `in_reply_to_user_id`**

In [123]:
df_api_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1746 entries, 0 to 1819
Data columns (total 10 columns):
tweet_id                 1746 non-null int64
created_at               1746 non-null datetime64[ns, UTC]
in_reply_to_status_id    22 non-null float64
in_reply_to_user_id      22 non-null float64
is_quote_status          1746 non-null bool
retweet_count            1746 non-null int64
favorite_count           1746 non-null int64
jpg_url                  1746 non-null object
retweeted                1746 non-null bool
favorited                1746 non-null bool
dtypes: bool(3), datetime64[ns, UTC](1), float64(2), int64(3), object(1)
memory usage: 114.2+ KB


# Dataframe principal

### Tweets archive

In [124]:
df_arch_clean.sample(2)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,text,rating_numerator,rating_denominator,name,category
559,803276597545603072,NaN,NaN,2016-11-28 16:37:19+00:00,This is Winston. His selfie game is legendary....,11,10,Winston,NaN
912,757596066325864448,NaN,NaN,2016-07-25 15:19:12+00:00,Here's another picture without a dog in it. Id...,4,10,NaN,NaN


### Twitter API

In [125]:
df_api_clean.sample(2)

,tweet_id,created_at,in_reply_to_status_id,in_reply_to_user_id,is_quote_status,retweet_count,favorite_count,jpg_url,retweeted,favorited
1736,667495797102141441,2015-11-20 00:12:54+00:00,NaN,NaN,False,272,517,http://pbs.twimg.com/media/CUNr4-7UwAAg2lq.jpg,False,False
1793,666373753744588802,2015-11-16 21:54:18+00:00,NaN,NaN,False,88,183,http://pbs.twimg.com/media/CT9vZEYWUAAlZ05.jpg,False,False


### Image prediction

In [126]:
df_predict_clean.sample(5)

,tweet_id,jpg_url,prediction_try,prediction_breed,confidence,is_a_dog
1283,750868782890057730,https://pbs.twimg.com/media/CmufLLsXYAAsU0r.jpg,1,toy poodle,0.912648,True
4189,669749430875258880,https://pbs.twimg.com/media/CUttjYtWcAAdPgI.jpg,3,paper towel,0.105664,False
5602,812466873996607488,https://pbs.twimg.com/media/C0Z2T_GWgAAxbL9.jpg,3,great dane,0.078205,True
5805,848324959059550208,https://pbs.twimg.com/media/C8XbDR1WAAAxND8.jpg,3,eskimo dog,0.154421,True
591,679475951516934144,https://pbs.twimg.com/media/CW37xZbUoAAUXe5.jpg,1,maltese dog,0.145742,True


In [127]:
df_predict_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5982 entries, 0 to 5981
Data columns (total 6 columns):
tweet_id            5982 non-null int64
jpg_url             5982 non-null object
prediction_try      5982 non-null int64
prediction_breed    5982 non-null object
confidence          5982 non-null float64
is_a_dog            5982 non-null bool
dtypes: bool(1), float64(1), int64(2), object(2)
memory usage: 286.2+ KB


### Criação do *dataframe* final

União dos dataframes `df_arch_clean` e `df_api_clean`

In [128]:
df_twitter_master = pd.merge(df_arch_clean, df_api_clean.loc[:, ['tweet_id', 'retweet_count', 'favorite_count']], how='left', on=['tweet_id'])

União dos dataframes `df_twitter_master` e `df_predict_clean`

In [129]:
# Seleção dos registros marcados como cães identificados nas fotos
df_trusted_dogs = df_predict_clean.loc[(df_predict_clean.is_a_dog), ['tweet_id', 'prediction_breed']]

In [130]:
df_trusted_dogs.sample(5)

,tweet_id,prediction_breed
1861,859924526012018688,french bulldog
4349,672877615439593473,borzoi
2892,699801817392291840,irish setter
1215,743609206067040256,weimaraner
763,689143371370250240,english springer


In [131]:
# Mesclagem do dataframe master com a seleção de predições confiáveis de cães
df_twitter_master = pd.merge(df_twitter_master, df_trusted_dogs, how='left', on=['tweet_id'])

#### Teste: Verificar dataframe final

In [132]:
df_twitter_master.sample(5)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,text,rating_numerator,rating_denominator,name,category,retweet_count,favorite_count,prediction_breed
2254,725842289046749185,NaN,NaN,2016-04-29 00:21:01+00:00,This is Colby. He's currently regretting all t...,12,10,Colby,NaN,2789.0,7298.0,toy poodle
2177,732585889486888962,NaN,NaN,2016-05-17 14:57:41+00:00,"When your teacher agreed on 10,000 RTs and no ...",10,10,NaN,NaN,815.0,3811.0,american staffordshire terrier
234,872122724285648897,NaN,NaN,2017-06-06 16:07:15+00:00,This is Kody. He's a baller. Wishes he was a l...,12,10,Kody,NaN,NaN,NaN,pug
3131,689661964914655233,NaN,NaN,2016-01-20 04:13:20+00:00,Meet Luca. He's a Butternut Scooperfloof. Glor...,12,10,Luca,NaN,984.0,3335.0,chihuahua
2267,724771698126512129,NaN,NaN,2016-04-26 01:26:53+00:00,Nothin better than a doggo and a sunset. 11/10...,11,10,NaN,doggo,675.0,2458.0,german short-haired pointer


In [133]:
df_twitter_master.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4907 entries, 0 to 4906
Data columns (total 12 columns):
tweet_id                 4907 non-null int64
in_reply_to_status_id    102 non-null float64
in_reply_to_user_id      102 non-null float64
timestamp                4907 non-null datetime64[ns, UTC]
text                     4907 non-null object
rating_numerator         4907 non-null int64
rating_denominator       4907 non-null int64
name                     3556 non-null object
category                 769 non-null object
retweet_count            4107 non-null float64
favorite_count           4107 non-null float64
prediction_breed         4418 non-null object
dtypes: datetime64[ns, UTC](1), float64(4), int64(3), object(4)
memory usage: 498.4+ KB


Conversão das colunas `in_reply_to_status_id`, `in_reply_to_user_id`, `retweet_count` e `favorite_count` para `pd.Int64Dtype()`

In [134]:
df_twitter_master.loc[:, 'in_reply_to_status_id'] = df_twitter_master.loc[:, 'in_reply_to_status_id'].astype(pd.Int64Dtype())
df_twitter_master.loc[:, 'in_reply_to_user_id'] = df_twitter_master.loc[:, 'in_reply_to_user_id'].astype(pd.Int64Dtype())

In [135]:
df_twitter_master.loc[:, ['retweet_count', 'favorite_count']] = df_twitter_master.loc[:, ['retweet_count', 'favorite_count']].astype(pd.Int64Dtype())

In [136]:
df_twitter_master.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4907 entries, 0 to 4906
Data columns (total 12 columns):
tweet_id                 4907 non-null int64
in_reply_to_status_id    102 non-null Int64
in_reply_to_user_id      102 non-null Int64
timestamp                4907 non-null datetime64[ns, UTC]
text                     4907 non-null object
rating_numerator         4907 non-null int64
rating_denominator       4907 non-null int64
name                     3556 non-null object
category                 769 non-null object
retweet_count            4107 non-null Int64
favorite_count           4107 non-null Int64
prediction_breed         4418 non-null object
dtypes: Int64(4), datetime64[ns, UTC](1), int64(3), object(4)
memory usage: 517.5+ KB


------
<a id="armazenamento"></a>

# Armazenamento

Salvar os dataframes limpos

In [137]:
data_folder_path = 'data'

In [138]:
# Dataframe clean: Twitter Archive
df_arch_clean.to_csv(os.path.join(data_folder_path, 'twitter_archive_clean.csv'), index=None, header=True)

In [139]:
# Dataframe clean: Tweet image prediction
df_predict_clean.to_csv(os.path.join(data_folder_path, 'tweet_image_prediction_clean.csv'), index=None, header=True)

In [140]:
# Dataframe clean: API requested Tweets
df_api_clean.to_csv(os.path.join(data_folder_path, 'twitter_api_clean.csv'), index=None, header=True)

In [141]:
# Dataframe Principal: Twitter Master 
df_twitter_master.to_csv(os.path.join(data_folder_path, 'twitter_archive_master.csv'), index=None, header=True)

# Relatórios

* Data wrangling efforts
* Analyses and visualizations